In [2]:
import torch
from torch.utils.data import DataLoader, random_split
from torch import nn
from torchvision.transforms import Normalize, Compose, ToTensor

import torchvision.models as models

import matplotlib.pyplot as plt
import numpy as np
import sys, time
import os, psutil

import pisco
import pisco.datasets
import tensorflow_datasets as tfds

/local/home/flanusse/.local/lib/python3.8/site-packages/pandas/core/computation/expressions.py:20: UserWarning: Pandas requires version '2.7.3' or newer of 'numexpr' (version '2.7.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


In [7]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [8]:
N_pix = 352

In [9]:
seed = 12345
torch.manual_seed(seed)

num_epochs = 5
batch_size = 64
learning_rate = 0.001

In [10]:
# use resnet18
model = models.resnet18()
model.fc = nn.Linear(512,2) # modif last line
model.conv1 = nn.Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)#modif first line

In [11]:
# send to GPU
model = model.to(device)

In [12]:
# define the optimization
criterion = torch.nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [14]:
dset = tfds.load('pisco_euclid',split='train')
dset = dset.repeat()
dset = dset.shuffle()
dset = dset.batch(batch_sizech_size)
dset = dset.as_numpy_iterator()

DatasetNotFoundError: Dataset pisco_euclid not found.
Available datasets:
	- abstract_reasoning
	- accentdb
	- aeslc
	- aflw2k3d
	- ag_news_subset
	- ai2_arc
	- ai2_arc_with_ir
	- amazon_us_reviews
	- anli
	- answer_equivalence
	- arc
	- asqa
	- asset
	- assin2
	- bair_robot_pushing_small
	- bccd
	- beans
	- bee_dataset
	- beir
	- big_patent
	- bigearthnet
	- billsum
	- binarized_mnist
	- binary_alpha_digits
	- ble_wind_field
	- blimp
	- booksum
	- bool_q
	- bucc
	- c4
	- c4_wsrs
	- caltech101
	- caltech_birds2010
	- caltech_birds2011
	- cardiotox
	- cars196
	- cassava
	- cats_vs_dogs
	- celeb_a
	- celeb_a_hq
	- cfq
	- cherry_blossoms
	- chexpert
	- cifar10
	- cifar100
	- cifar100_n
	- cifar10_1
	- cifar10_corrupted
	- cifar10_n
	- citrus_leaves
	- cityscapes
	- civil_comments
	- clevr
	- clic
	- clinc_oos
	- cmaterdb
	- cnn_dailymail
	- coco
	- coco_captions
	- coil100
	- colorectal_histology
	- colorectal_histology_large
	- common_voice
	- conll2002
	- conll2003
	- controlled_noisy_w

In [ ]:
##########################################################################
### TRAIN MODEL ###

print('Train loader size = ', len(train_loader))
print('Validation loader size = ',len(val_loader))

loss_list = {'train':[], 'val':[]}
acc_list = {'train':[], 'val':[]}

print('\n-------------------------------------------------')
print('TRAIN MODEL\n')
for epoch in range(num_epochs):
    # enumerate mini batches
    for phase in ['train']:
        if phase == "train":
            loss_total = 0
            acc_total = 0
            for i, (X, Y) in enumerate(dset):

                # send to GPU
                X = X.to(device)
                Y = Y.to(device)

                # clear the gradients
                optimizer.zero_grad()

                # compute the model output
                Yhat = model(X.float())
                #print(Yhat,Y)
                #print(Y[:,:2])

                # calculate loss
                loss = criterion(Yhat.float(), Y[:,:2].float())
                #loss = criterion(Yhat.float(), Y.long())

                # credit assignment
                loss.backward()

                # save the loss to check progress
                loss_total += loss.detach()

                # update model weights
                optimizer.step()

                # Accuracy for regression
                # MAE (mean absolute error) between prediction and true S8 
                acc = torch.nn.L1Loss()(Yhat.float(),Y[:,:2].float())
                acc_total += acc.detach()

            loss_list[phase].append(loss_total.cpu()/len(train_loader))
            acc_list[phase].append(acc_total.cpu()/len(train_loader))

        else:
            loss_total = 0
            acc_total = 0
            for i, (X, Y) in enumerate(val_loader):

                # send to GPU
                X = X.to(device)
                Y = Y.to(device)
                # compute the model output
                Yhat = model(X.float())

                # calculate loss
                loss = criterion(Yhat.float(), Y[:,:2].float())
                #loss = criterion(Yhat.float(), Y.long())

                # save the loss to check progress
                loss_total += loss.detach()
                #loss_list[phase].append(loss.item())

                # Accuracy for regression
                # MAE (mean absolute error) between prediction and true S8 
                acc = torch.nn.L1Loss()(Yhat.float(),Y[:,:2].float())
                acc_total += acc.detach()
                #acc_list[phase].append(acc)

            loss_list[phase].append(loss_total.cpu()/len(val_loader))
            acc_list[phase].append(acc_total.cpu()/len(val_loader))

    print('Epoch [{}/{}], Loss_train: {:.4f}, Loss_val: {:.4f}, Accuracy_train: {:.2f}, Accuracy_val: {:.2f}'.format(epoch + 1, num_epochs, loss_list['train'][epoch].item(), loss_list['val'][epoch].item(), acc_list['train'][epoch].item(), acc_list['val'][epoch].item()))